In [1]:
from multitask_dataset import SingerMultiTaskDataset
from torch.utils.data import DataLoader
from torch.optim import Adam
from models import HuBERTMultiHead
import pandas as pd

import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

from torchview import draw_graph
import torchvision
# from torchviz import make_dot
from torchsummary import summary

# import torch
# torch.autograd.detect_anomaly(True)

In [2]:
train_audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/'
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/multitask_targets.csv'

In [3]:
feats = pd.read_csv(csv_path, delimiter=',')
features_list = list(feats.columns)
print(features_list)
del(features_list[:2])
print(features_list)

['Unnamed: 0', 'names', 'singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'Rate', 'Depth', 'Regularity']
['singer_id', 'Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', '

In [4]:
task_labels_num_out = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_num_out[features_list[i]] = 1
# add singer identification
task_labels_num_out['singer_id'] = feats['singer_id'].max()+1 # accounting for zero
print(task_labels_num_out)

{'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'singer_id': 6}


In [5]:
model = HuBERTMultiHead(task_labels_num_out=task_labels_num_out)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
training_data = SingerMultiTaskDataset(train_audio_folder, csv_path)

In [7]:
print(len(training_data))

336


In [8]:
print(training_data[235])

{'input_values': array([ 9.62317863e-05, -4.80651506e-04, -1.05629163e-03, -1.39596628e-03,
       -1.47772674e-03, -8.91393982e-04, -9.50069734e-05, -1.73296430e-05,
       -6.29215094e-04, -4.58008086e-04,  3.03915498e-04,  7.24024838e-04,
       -3.85877094e-04, -1.58438459e-03, -2.10130448e-03, -2.19018199e-03,
       -1.65093027e-03, -1.03976380e-03, -4.16553317e-04, -5.17049571e-04,
       -6.52124872e-04, -3.58587335e-04, -1.37439056e-04,  1.60662661e-04,
        6.28071837e-04,  1.10055821e-03,  1.16100477e-03,  1.34328124e-03,
        1.92774879e-03,  2.54732207e-03,  2.86061224e-03,  2.78626406e-03,
        2.24109413e-03,  1.05919642e-03,  3.76034004e-04,  1.56268405e-04,
        2.39309418e-04,  2.05230288e-04, -2.34994695e-05, -5.72757679e-04,
       -1.73673185e-03, -2.30674958e-03, -2.20199744e-03, -2.12050648e-03,
       -1.97671400e-03, -1.42902718e-03, -1.12089305e-03, -1.28791505e-03,
       -1.52374781e-03, -1.03508693e-03, -1.70338928e-04, -2.10182974e-04,
       -

In [9]:
dataloader = DataLoader(training_data, batch_size=4, shuffle=True, collate_fn=model.collate_fn)

In [10]:
print(dataloader.__dict__)

{'dataset': <multitask_dataset.SingerMultiTaskDataset object at 0x7adbdd98f440>, 'num_workers': 0, 'prefetch_factor': None, 'pin_memory': False, 'pin_memory_device': '', 'timeout': 0, 'worker_init_fn': None, '_DataLoader__multiprocessing_context': None, '_dataset_kind': 0, 'batch_size': 4, 'drop_last': False, 'sampler': <torch.utils.data.sampler.RandomSampler object at 0x7adb4e7a8d10>, 'batch_sampler': <torch.utils.data.sampler.BatchSampler object at 0x7adbb1e7c110>, 'generator': None, 'collate_fn': <bound method HuBERTMultiHead.collate_fn of HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
      

In [11]:
b = next(iter(dataloader))

In [12]:
print(len(b[0]['input_values']))
print(len(b[0]['attention_mask']))
print(len(b[1]))
print(b[1])

4
4
33
{'Unnamed: 0': [102, 242, 85, 149], 'names': ['kazantzidis_e_9', 'keti_grei_i_4', 'kazantzidis_a_3', 'kazantzidis_old_i_3'], 'singer_id': [2, 3, 2, 2], 'Pitch': [0.05629582318327267, 0.8383463052707324, 0.229312249629672, 0.25344539783365794], 'SpectralCentroid': [0.45576782982821595, 0.7904943151772378, 0.43304689864132573, 0.28658310260291936], 'SpectralSpread': [0.5179841764941729, 0.5501992340764359, 0.3112823945753109, 0.44008240475114746], 'SpectralSkewness': [0.25094154583215833, 0.1635885931786989, 0.5206666620311641, 0.3094674193166383], 'SpectralKurtosis': [0.050642980100993246, 0.05916134413650475, 0.2329566033048223, 0.07104226679105485], 'SpectralFlatness': [2.269617137682595e-05, 0.00014026705486352474, 0.0014159502214822954, 0.14944453156773174], 'SpectralCrest': [0.045173819853207856, 0.15336599232934336, 0.22897452928015088, 0.4519815762066995], 'SpectralSlope': [0.5012307792889056, 0.4982146822036268, 0.46867066605553087, 0.2547711636207461], 'SpectralDecrease'

In [13]:
# model.train()
# model.eval()
for p in model.hubert.parameters():
    p.requires_grad = False
train_params = []
for k in model.projectors.keys():
    train_params.extend( list(model.projectors[k].parameters()) )
    train_params.extend( list(model.classifiers[k].parameters()) )
    print(k)
    model.projectors[k].requires_grad = True
    model.classifiers[k].requires_grad = True
# # model.train(False)
# print(train_params)
# optimizer = Adam( model.classifiers['Pitch'].parameters() )
optimizer = Adam( model.parameters(), lr=0.001 )

Pitch
SpectralCentroid
SpectralSpread
SpectralSkewness
SpectralKurtosis
SpectralFlatness
SpectralCrest
SpectralSlope
SpectralDecrease
SpectralRollOff
SpectralVariation
SpectralFlux
HarmonicSpectralDeviation
Tristimulus_1
Tristimulus_2
Tristimulus_3
HarmonicOddToEvenRatio
Inharmonicity
HarmonicEnergy
NoiseEnergy
Noisiness
HarmonicToNoiseEnergy
PartialsToNoiseEnergy
F1_Hz
F2_Hz
F3_HZ
F4_Hz
singer_id


In [14]:
# for p in model.parameters():
#     print(p)
print(model.parameters())

<generator object Module.parameters at 0x7adb4e5b0820>


In [15]:
print(optimizer)

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


In [16]:
print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None


In [17]:
b[0]['attention_mask']

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [38]:
print(b[1]['singer_id'])

[2, 3, 2, 2]


In [18]:
y = model(
    audio_normalized=b[0]['input_values'],
    attention_mask=b[0]['attention_mask'],
    labels=b[1],
    output_attentions=False,
    output_hidden_states=False,
    return_dict=True
)

In [19]:
print(y)

SequenceClassifierOutput(loss=tensor(3.9053, device='cuda:0', grad_fn=<AddBackward0>), logits={'Pitch': [tensor([[0.4806],
        [0.4778],
        [0.4693],
        [0.5004]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCentroid': [tensor([[0.5077],
        [0.5028],
        [0.4996],
        [0.5209]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSpread': [tensor([[0.5005],
        [0.4959],
        [0.4992],
        [0.5063]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralSkewness': [tensor([[0.4709],
        [0.4709],
        [0.4675],
        [0.4699]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralKurtosis': [tensor([[0.5006],
        [0.4950],
        [0.4958],
        [0.4954]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralFlatness': [tensor([[0.5312],
        [0.5288],
        [0.5204],
        [0.5429]], device='cuda:0', grad_fn=<SigmoidBackward0>)], 'SpectralCrest': [tensor([[0.4920],
        [0.4906],
        [0.4880],


In [20]:
print(y['logits'].keys())
for k in y['logits'].keys():
    print(y['logits'][k])

dict_keys(['Pitch', 'SpectralCentroid', 'SpectralSpread', 'SpectralSkewness', 'SpectralKurtosis', 'SpectralFlatness', 'SpectralCrest', 'SpectralSlope', 'SpectralDecrease', 'SpectralRollOff', 'SpectralVariation', 'SpectralFlux', 'HarmonicSpectralDeviation', 'Tristimulus_1', 'Tristimulus_2', 'Tristimulus_3', 'HarmonicOddToEvenRatio', 'Inharmonicity', 'HarmonicEnergy', 'NoiseEnergy', 'Noisiness', 'HarmonicToNoiseEnergy', 'PartialsToNoiseEnergy', 'F1_Hz', 'F2_Hz', 'F3_HZ', 'F4_Hz', 'singer_id'])
[tensor([[0.4806],
        [0.4778],
        [0.4693],
        [0.5004]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5077],
        [0.5028],
        [0.4996],
        [0.5209]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5005],
        [0.4959],
        [0.4992],
        [0.5063]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.4709],
        [0.4709],
        [0.4675],
        [0.4699]], device='cuda:0', grad_fn=<SigmoidBackward0>)]
[tensor([[0.5006],
    

In [21]:
optimizer.zero_grad()
y.loss.backward()# retain_graph=True
optimizer.step()

In [22]:
print(model)

HuBERTMultiHead(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(


In [23]:
print(model.classifiers['Pitch'].weight.grad)
# print(model.projectors['F4_Hz'].weight)

tensor([[ 2.0181e-03, -2.3428e-04,  2.0888e-03, -7.2412e-05, -1.8582e-04,
          1.4629e-02,  1.4761e-02,  2.0537e-03,  8.4879e-03,  2.9768e-03,
          1.3497e-03,  4.2776e-03, -1.7067e-04,  3.8212e-03,  4.6032e-03,
          3.1270e-03, -6.2038e-05,  1.2604e-02, -2.3571e-05,  1.9151e-03,
         -3.9052e-05,  4.4018e-03, -2.0840e-04,  1.8382e-02,  1.0132e-02,
          4.7192e-03,  5.3779e-03,  1.4175e-04,  1.3236e-02, -1.1823e-04,
          7.5141e-03,  4.5607e-03,  4.9791e-03,  6.3905e-03,  3.9887e-03,
         -4.0342e-05, -2.4124e-05,  9.2060e-03,  1.7273e-03,  2.1391e-03,
         -1.2108e-04,  7.3856e-03,  1.7027e-03,  1.4694e-03,  4.9782e-03,
          2.0304e-03, -9.2368e-05,  9.7770e-05,  3.1595e-03, -6.9385e-05,
         -1.0797e-04,  2.4465e-03,  3.7246e-03,  8.8016e-03, -1.2359e-04,
          3.3121e-03,  4.7053e-03,  4.5873e-03,  1.6720e-03,  1.1553e-03,
          9.1494e-04,  1.2501e-02,  3.7191e-03,  7.5407e-04]], device='cuda:0')


In [24]:
print(model.hubert.encoder.layers[11].final_layer_norm.weight.grad)
print(model.hubert.encoder.layers[11].final_layer_norm.weight)

None
Parameter containing:
tensor([0.2295, 0.3408, 0.2241, 0.2180, 0.3718, 0.2739, 0.4177, 0.7271, 0.2072,
        0.2150, 0.2554, 0.2473, 0.2111, 0.7178, 0.1851, 0.2050, 0.4814, 0.2057,
        0.2379, 0.1952, 0.5957, 0.3611, 0.2522, 0.5278, 0.1882, 1.0547, 0.2152,
        0.4490, 0.4299, 0.2314, 0.2019, 0.2072, 0.2798, 0.2083, 0.4707, 0.2754,
        0.2136, 0.6519, 0.2372, 0.2583, 0.2527, 0.2141, 0.2001, 0.3301, 0.2196,
        0.2537, 0.2581, 0.2908, 0.2773, 0.2332, 0.1733, 0.2766, 0.2544, 0.2703,
        0.2329, 0.2493, 0.2891, 0.2959, 0.2852, 0.6724, 0.2040, 0.2484, 0.2087,
        0.3169, 0.2595, 0.1816, 0.2347, 0.7002, 0.2201, 0.2742, 0.2114, 0.2729,
        0.6841, 0.6235, 0.4590, 0.1656, 0.2288, 0.4587, 0.3438, 0.2225, 0.4719,
        0.2688, 0.2349, 0.2260, 0.2186, 0.2642, 0.2301, 0.3638, 0.2355, 0.4534,
        0.1997, 0.1818, 0.1923, 0.2277, 0.7339, 0.2678, 0.2124, 0.2478, 0.2356,
        0.3901, 0.2098, 0.2205, 0.2212, 0.6611, 0.2272, 0.2209, 0.2822, 0.2277,
        0.570

In [25]:
print(len(model.hubert.encoder.layers))

12


In [26]:
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight.grad)
print(model.hubert.encoder.layers[11].feed_forward.output_dense.weight)
# print(model.hubert.encoder.layers[0].attention.k_proj.weight.grad)

None
Parameter containing:
tensor([[-0.0309, -0.0560, -0.0944,  ..., -0.0551,  0.0054, -0.1141],
        [-0.0452,  0.0558, -0.0271,  ..., -0.0301, -0.0216,  0.0928],
        [ 0.0855, -0.0440,  0.1083,  ..., -0.0047,  0.0585, -0.1077],
        ...,
        [-0.0461, -0.0809,  0.0314,  ...,  0.0263,  0.0117,  0.0425],
        [-0.0258, -0.0419, -0.0305,  ..., -0.0015, -0.0509, -0.1317],
        [-0.0070, -0.0095,  0.0326,  ...,  0.0144,  0.0219, -0.0197]],
       device='cuda:0')


In [30]:
for i in range(50):
    y = model(
        audio_normalized=b[0]['input_values'],
        attention_mask=b[0]['attention_mask'],
        labels=b[1],
        output_attentions=False,
        output_hidden_states=False,
        return_dict=True
    )
    print(y.loss)
    # print(model.classifiers['F4_Hz'].weight)
    optimizer.zero_grad()
    y.loss.backward()
    optimizer.step()

tensor(1.0465, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0464, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0463, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0463, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0462, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0462, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0461, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0461, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0461, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0460, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0460, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0459, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0459, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0458, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0458, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0458, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0457, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0457, device='cuda:0', grad_fn=<AddBack

In [42]:
print(y['logits']['singer_id'][0].argmax(dim=1, keepdim=True).squeeze())

tensor([2, 3, 2, 2], device='cuda:0')
